# AWRDE Tutorial
***

Just a simple tutorial for working with the AWRDE API.

## Imports
***
Import what you need:

In [1]:
import pyawr.mwoffice as mwo
import numpy as np
import matplotlib.pyplot as plt

#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl package.
%matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter package.
# %matplotlib tk 

## First steps:
***

We need to create an AWRDE object so that we can refer to it when we work with the API.  (Make sure your AWRDE project is open.)

NOTE: if you have several versions of AWRDE, you should specify which one you want to work with: `awrde = mwo.CMWOffice(version='16.0')`

In [2]:
awrde = mwo.CMWOffice() #Create awrde object

Let's test if we can see the names of things in our AWR project.

In [3]:
# Things in AWRDE are indexed (starting with (1) if using parentheses, and starting with [0] if using brackets. 
# We look in the awrde object for the Project that's open. 
# We look in the Project for its Graphs. 
# We look in the Graphs for graph number 2, i.e Item(2).
# Item number 2 in Graphs has a name, and we can use this name to refer to it. We'll call it graph_name:
graph_name = awrde.Project.Graphs.Item(2).Name

# The graph itself can be assigned to the variable 'graph' as follows:
graph = awrde.Project.Graphs(graph_name)

# We can get the actual measurements, i.e. the trace from the graph, as follows.
# The following line will assign the first measurement in the graph to 'meas'.
meas = graph.Measurements[0]

# We print the project name to see if it works
print(awrde.Project.Name)

# We can also
print(graph.Name)
print(meas.Name)
# Etc.
# Code completion suggestions are helpful here.

Tutorial.emp
Graph_Hanger
Hanger_Mode:DB(|S(2,1)|)


The following command is necessary for running the actual analysis in AWRDE. You should get the AWRDE analysis popup after executing the following cell: 

![Simulation window](./SimulationWindow.png)


In [4]:
awrde.Project.Simulator.Analyze() #Invoke circuit simulator analysis

## Plotting With Numpy:

***

We can plot the various "measurements" from AWRDE pretty easily:

In [5]:
fig = plt.figure()
ax = plt.subplot(1,1,1)

# The following can be a useful API call to know, but in general 
# this kind of variable initialization is not very useful. 
num_pts = meas.XPointCount 
xs = ys = np.zeros(num_pts)

xs = meas.XValues
ys = meas.YValues(1)

ax.plot(xs, ys)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

That's about it for basic functionality of the API that I wanted to show. As I mentioned before, code completion is probably the most useful tool for getting used to it. That said, there's an API reference in the same folder as this notebook with more detailed coverage of all the functionality.

# Extra Stuff:
## Animations With Numpy:

***

While working with AWR and Python, I found myself rerunning `awrde.Project.Simulator.Analyze()` many times as I messed with circuit values in AWR. This seemed unideal so I came up with a "better" solution...  

Now, if I wanted to see graphs change in real-time when messing with the tuner in AWRDE for example, I could do the following:

Add this import statement. (You can read more about matplotlib's [FuncAnimation](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html) online.)

In [6]:
from matplotlib.animation import FuncAnimation 

### Some Jupyter magic:
Actually, this doesn't fully help us animate things in Jupyter notebooks, which normally render static images when you plot things. 

Enabling AGG rendering required some "magic," and even then it is kinda buggy. That said, matplotlib can work with several [backends](https://matplotlib.org/stable/tutorials/introductory/usage.html#the-builtin-backends) that enable this functionality.

 [**ipympl**](https://github.com/matplotlib/ipympl):  
 ipympl will render animations inline in a jupyter notebook. It can be buggy and sometimes cell execution hangs, requiring a restart of the python kernel.

 ```conda install -c conda-forge ipympl```

 [**tkinter**](https://docs.python.org/3/library/tkinter.html):  
 tkinter will render graphs in a seperate window. This is useful when you want to plot a lot of stuff at once.

 ```conda install tk```

 They won't both work together, so one needs to be picked. Also since this is weird Jupyter magic you can't have inline comments. 

 * Bad:  
   `%matplotlib ipympl     # magical magic`  
 &nbsp;
 * Good:   
  `# magical`  
  `%matplotlib ipympl`  
  `# magic`  

Now, if you enable the tuner tool in AWRDE and move the values around you should see the graph below change.

**NOTE: *This line needs to remain immediately after the import statements for it to work,*** that's why it's where it is. If you want to see the graphs in a seperate window, rerun the code with the ipympl commented out and the tk line uncommented:

```
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl package
#%matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter package
%matplotlib tk 
```

In [7]:
num_pts = meas.XPointCount
xs = ys = np.zeros(num_pts)

def animate(n_frm): #Animate method for FuncAnimation
    xs = np.asarray(meas.XValues)
    ys = np.asarray(meas.YValues(1))

    # trace.set_data(xs, ys)

    axx.cla()
    axx.plot(xs, ys, 'r')

    # return [trace] #Necessary to return Artist objects if blit=True in FuncAnimation

figg = plt.figure()
axx = plt.subplot(1,1,1)

ani = FuncAnimation(figg, animate, interval=200, blit=False) #Create animation object. blit=True is for smoother animations, only changed data should be updated.
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Other Miscellaneous Stuff:

***

### Listing Schematics

In [8]:
NumSchem = awrde.Project.Schematics.Count

for s_id in range(NumSchem):
    schem = awrde.Project.Schematics[s_id]
    print(schem.Name)

Hanger_Mode
Parallel_Mode
Sample_Subcircuit


### Resetting Frequencies

In [9]:
def reset_freqs (l_bnd=7e9, u_bnd=8.6e9, steps=10000):
    '''
    Reset the MWO project frequencies.

    returns an array running from 
    lower_bound to upper_bound in steps steps.
    '''
    
    awrde.Project.Frequencies.Clear() # clear the frequencies specified for the project
    freq_arr = np.linspace(l_bnd, u_bnd, steps)
    awrde.Project.Frequencies.AddMultiple(freq_arr) # add the frequencies in the passed frequency array

    awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return freq_arr

reset_freqs

<function __main__.reset_freqs(l_bnd=7000000000.0, u_bnd=8600000000.0, steps=10000)>

### Setting Circuit Paramters
 * The first Schematic is 'Hanger_Mode'
    * Its first element is 'CAP.C2'
        * 'CAP.C2''s second parameter is 'C' (capacitance.) (The first one is 'ID'.)

In [10]:
awrde.Project.Schematics(1).Elements(1).Parameters(2).ValueAsDouble = 5.43e-13

### Getting Circuit Paramters

In [11]:
awrde.Project.Schematics(1).Elements(1).Parameters(2).ValueAsDouble

5.43e-13

## Conclusion: 
***
This about covers the useful things that I learned how to do. See the other files for more not-so-neat examples of what can be done. `circle_fit_sandbox.ipynb` in the `../ResonatorSensitivity` directory might have the best examples.